# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [47]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [48]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [49]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# getting total number of rows 
print('Total number of rows: ', len(full_data_rows_list))
# checking to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total number of rows:  8056


In [50]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion  

## Now ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [51]:
# This should make a connection to a Cassandra instance of the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [52]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [53]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

1. Give me the **artist**, **song title** and **song's length** in the music app history that was heard during  **sessionId = 338**, and **itemInSession  = 4**.

2. Give me only the following: **name of artist**, **song (sorted by itemInSession)** and **user (first and last name)** for **userid = 10**, **sessionid = 182**.
    
3. Give me every **user name (first and last)** in my music app history who listened to the **song** 'All Hands Against His Own'.

#### 1. Give me the **artist**, **song title** and **song's length** in the music app history that was heard during  **sessionId = 338**, and **itemInSession  = 4**.

In [54]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, \
PRIMARY  KEY(session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

> I chose session_id and item_in_session as composite primary key because the query uses them in this order. In this case, item_in_session is a clustering column that sorts within session_id. This helps to optimize the performance of the query. 

In [55]:
# process the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        #  Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [56]:
# Add in the SELECT statement to verify the data was entered into the table
query = ("""SELECT artist_name, song_title, song_length FROM music_library
WHERE session_id = 338 and item_in_session = 4 """)

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


> The query returns one record consists of artist name, song title and song length where session_id = 338 and item_in_session= 4.

#### 2. Give me only the following: **name of artist**, **song (sorted by itemInSession)** and **user (first and last name)** for **userid = 10**, **sessionid = 182**.

In [39]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
session.execute("DROP TABLE IF EXISTS music_library")

query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text,  first_name text, last_name text,  \
 PRIMARY  KEY((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

> In this table, I chose user_id and session_id as a composite primary key, and item_in_session clustering column. The query searches by user_id first, and session_id second. That means within the same user_id, it will look for session_id, and it cuts down search load and time. Session_id is a clustering column that is also a part of the primary key. I used item_in_session as a clustering column in order to sort songs as requested in the query. 

In [40]:
# process the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [41]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, first_name, last_name FROM music_library WHERE user_id = 10 and session_id = 182"
    
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


> The query on the table returns four records that contains artist name, song title, first and last name of users where user_id = 10 and session_id = 182. The records are sorted by item_in_session.

#### 3. Give me every **user name (first and last)** in my music app history who listened to the **song** 'All Hands Against His Own'.

In [42]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("DROP TABLE IF EXISTS music_library")

query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(song_title text, user_id int, first_name text, last_name text,  PRIMARY KEY((song_title), user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)                     

> For this query, I chose song_title as a primary key and user_id as a clustering column. The query searched the records by a song title. Therefore, setting the song title as a primary key is  the best option. Query results also should display user first and last name. I chose user_id as a clustering column to sort user first and last name by user_id once song title matches the queried song title.

In [43]:
# process the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [44]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT  first_name, last_name FROM music_library WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


> The query results in three records of users' first and last time where users listened to a song titled "All Hands Agains His Own".

### Drop the tables before closing out the sessions

In [45]:
# Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS music_library")

### Close the session and cluster connection¶

In [46]:
session.shutdown()
cluster.shutdown()